# Toggling Objective — IPOPT Resource Analysis

imports

In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();
Pkg.develop(path="../../QuantumCollocation.jl")
# Pkg.develop(path="../../Piccolissimo.jl")
using PiccoloQuantumObjects
using QuantumCollocation
using ForwardDiff
using LinearAlgebra
using Plots
using SparseArrays
using Statistics
using CairoMakie
using NamedTrajectories
using TrajectoryIndexingUtils
using Random
using CSV
using Measures: mm
using Random
using FilePathsBase, FileIO
using DataFrames


  Activating project at `~/Documents/research/pulses/project/notebooks/src`
┌ Warning: Circular dependency detected.
│ Precompilation will be skipped for dependencies in this cycle:
│  ┌ Piccolissimo
│  └─ QuantumCollocation
└ @ Base.Precompilation precompilation.jl:651
   Resolving package versions...
  No Changes to `~/Documents/research/pulses/project/notebooks/src/Project.toml`
  No Changes to `~/Documents/research/pulses/project/notebooks/src/Manifest.toml`
┌ Warning: Circular dependency detected.
│ Precompilation will be skipped for dependencies in this cycle:
│  ┌ Piccolissimo
│  └─ QuantumCollocation
└ @ Base.Precompilation precompilation.jl:651
┌ Warning: Replacing docs for `QuantumCollocation.ProblemTemplates.UnitaryToggleProblem :: Union{}` in module `QuantumCollocation.ProblemTemplates`
└ @ Base.Docs docs/Docs.jl:243
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
┌ Warning: Replacing docs fo

helper functions for tracking

In [2]:
mutable struct SolverMetrics
    wall_times::Vector{Float64}
    iter_times::Vector{Float64}
    objectives::Vector{Float64}
    constraints::Vector{Float64}
    memory_usage::Vector{Float64}
    iterations::Vector{Int}
    convergence_rates::Vector{Float64}
    SolverMetrics() = new([], [], [], [], [], [], [])
end



In [3]:
function track_solver_performance(prob; max_iter=150, print_level=5, convergence_threshold=1e-5)
    """
    Track detailed performance metrics during IPOPT solving
    """
    metrics = SolverMetrics()
    
    # Custom callback to track iteration data
    iteration_data = []
    prev_obj = Inf
    convergence_iter = nothing
    
    # Start timing
    start_time = time()
    start_memory = Base.gc_live_bytes() / 1024^2  # Convert to MB
    
    # Solve without custom callback first (IPOPT callback signature is complex)
    # We'll use a simpler approach: solve in chunks and track between solves
    
    # Run the optimization
    result = solve!(prob; 
        max_iter=max_iter, 
        print_level=print_level
    )
    
    end_time = time()
    total_time = end_time - start_time
    end_memory = Base.gc_live_bytes() / 1024^2
    
    # For now, we'll track aggregated metrics
    push!(metrics.wall_times, total_time)
    push!(metrics.memory_usage, end_memory - start_memory)
    
    return metrics, nothing, total_time, end_memory - start_memory
end

function track_solver_performance_detailed(prob; max_iter=150, print_level=5, convergence_threshold=1e-5, chunk_size=10)
    """
    Track detailed performance metrics during IPOPT solving by running in chunks
    """
    metrics = SolverMetrics()
    # initial_obj = Inf
    # Start timing
    start_time = time()
    start_memory = Base.gc_live_bytes() / 1024^2  # Convert to MB
    
    prev_obj = Inf
    convergence_iter = nothing
    total_iters = 0
    
    # Solve in chunks to track progress
    remaining_iters = max_iter
    while remaining_iters > 0
        chunk_iters = min(chunk_size, remaining_iters)
        chunk_start = time()
        
        initial_obj = Inf
        # Get initial objective value if available
        try
            Z_vec = vec(prob.trajectory)
            initial_obj = prob.objective.L(Z_vec)
        catch
            initial_obj = Inf
        end
        
        # Run optimization chunk
        result = solve!(prob; 
            max_iter=chunk_iters, 
            print_level=print_level
        )
        
        chunk_end = time()
        chunk_time = chunk_end - chunk_start
        curr_memory = Base.gc_live_bytes() / 1024^2
        
        # Get final objective value
        final_obj = initial_obj
        try
            Z_vec = vec(prob.trajectory)
            final_obj = prob.objective.L(Z_vec)
        catch
            final_obj = initial_obj
        end
        
        # Track metrics
        total_iters += chunk_iters
        push!(metrics.iterations, total_iters)
        push!(metrics.wall_times, chunk_end - start_time)
        push!(metrics.objectives, final_obj)
        push!(metrics.iter_times, chunk_time / chunk_iters)
        push!(metrics.memory_usage, curr_memory - start_memory)
        
        # Check convergence
        if prev_obj != Inf
            rate = abs(final_obj - prev_obj) / max(abs(prev_obj), 1e-10)
            push!(metrics.convergence_rates, rate)
            if !convergence_iter && rate < convergence_threshold
                convergence_iter = total_iters
            end
        end
        prev_obj = final_obj
        
        # Check if we've converged
        if convergence_iter
            break
        end
        
        remaining_iters -= chunk_iters
    end
    
    end_time = time()
    total_time = end_time - start_time
    end_memory = Base.gc_live_bytes() / 1024^2
    
    return metrics, convergence_iter, total_time, end_memory - start_memory
end


track_solver_performance_detailed (generic function with 1 method)

In [4]:
function analyze_convergence(metrics::SolverMetrics, threshold=1e-6)
    """
    Analyze convergence characteristics of the solver
    """
    # Handle empty metrics
    if isempty(metrics.iterations)
        return Dict(
            "convergence_iteration" => nothing,
            "total_iterations" => 0,
            "avg_iteration_time" => NaN,
            "std_iteration_time" => NaN,
            "total_wall_time" => 0.0,
            "peak_memory_mb" => 0.0,
            "final_objective" => NaN,
            "final_constraint_violation" => NaN
        )
    end
    
    convergence_iter = nothing
    for (i, rate) in enumerate(metrics.convergence_rates)
        if rate < threshold
            convergence_iter = i + 1  # +1 because rates start from iteration 2
            break
        end
    end
    
    # Handle empty iter_times
    avg_iter_time = isempty(metrics.iter_times) ? NaN : mean(metrics.iter_times)
    std_iter_time = isempty(metrics.iter_times) ? NaN : std(metrics.iter_times)
    
    # Handle empty collections with defaults
    total_wall_time = isempty(metrics.wall_times) ? 0.0 : metrics.wall_times[end]
    peak_memory = isempty(metrics.memory_usage) ? 0.0 : maximum(metrics.memory_usage)
    final_obj = isempty(metrics.objectives) ? NaN : metrics.objectives[end]
    final_constraint = isempty(metrics.constraints) ? NaN : metrics.constraints[end]
    
    return Dict(
        "convergence_iteration" => convergence_iter,
        "total_iterations" => length(metrics.iterations),
        "avg_iteration_time" => avg_iter_time,
        "std_iteration_time" => std_iter_time,
        "total_wall_time" => total_wall_time,
        "peak_memory_mb" => peak_memory,
        "final_objective" => final_obj,
        "final_constraint_violation" => final_constraint
    )
end


analyze_convergence (generic function with 2 methods)

In [5]:
# Problem parameters
T = 20
Δt = 0.2
U_goal = GATES.X
H_drive = [PAULIS.X, PAULIS.Y, PAULIS.Z]
piccolo_opts = PiccoloOptions(verbose=false)
pretty_print(X::AbstractMatrix) = Base.show(stdout, "text/plain", X);
sys = QuantumSystem(H_drive)


QuantumSystem: levels = 2, n_drives = 3

In [6]:
n_guesses = 3
n_drives = sys.n_drives

def_seeds = []
add_seeds = []
mult_seeds = []
both_seeds = []
n_seeds = n_guesses

# Initialize trajectories
for i in 1:n_seeds
    Random.seed!(1234+i)
    a_bounds = fill(1.0, n_drives)
    da_bounds = fill((9+i)*π*Δt/T, n_drives)
    dda_bounds = fill(1.0, n_drives)
    control_bounds = (a_bounds, da_bounds, dda_bounds)
    
    def_traj = initialize_trajectory(
        U_goal, T, Δt, n_drives, control_bounds; system=sys
    )
    push!(def_seeds, def_traj)
    
    add_traj = initialize_trajectory(
        U_goal, T, Δt, n_drives, control_bounds; system=sys
    )
    push!(add_seeds, add_traj)
    
    mult_traj = initialize_trajectory(
        U_goal, T, Δt, n_drives, control_bounds; system=sys
    )
    push!(mult_seeds, mult_traj)
    
    both_traj = initialize_trajectory(
        U_goal, T, Δt, n_drives, control_bounds; system=sys
    )
    push!(both_seeds, both_traj)
end

In [7]:
# Fidelity targets
a_vals = exp.(range(log(100), log(100000), length=10))
final_fid_floor_log = 1 .- 1 ./ a_vals
final_fid_floor = final_fid_floor_log

n_nines = length(final_fid_floor)
default_probs = Matrix{Any}(undef, n_seeds, n_nines)
init_add_probs = Matrix{Any}(undef, n_seeds, n_nines)
init_mult_probs = Matrix{Any}(undef, n_seeds, n_nines)
init_both_probs = Matrix{Any}(undef, n_seeds, n_nines)

init_def_fids = zeros(n_seeds, n_nines)
init_add_fids = zeros(n_seeds, n_nines)
init_mult_fids = zeros(n_seeds, n_nines)

3×10 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [8]:
init_both_probs

3×10 Matrix{Any}:
 #undef  #undef  #undef  #undef  #undef  …  #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef     #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef     #undef  #undef  #undef  #undef

resource tracking storage

In [9]:
# Create structures to store performance metrics
default_metrics = Matrix{Dict}(undef, n_seeds, n_nines)
add_metrics = Matrix{Dict}(undef, n_seeds, n_nines)
mult_metrics = Matrix{Dict}(undef, n_seeds, n_nines)
final_add_metrics = Matrix{Dict}(undef, n_seeds, n_nines)
final_mult_metrics = Matrix{Dict}(undef, n_seeds, n_nines)


3×10 Matrix{Dict}:
 #undef  #undef  #undef  #undef  #undef  …  #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef     #undef  #undef  #undef  #undef
 #undef  #undef  #undef  #undef  #undef     #undef  #undef  #undef  #undef

problem setup with resource tracking

In [10]:
Hₑ_add = a -> PAULIS.X
X_drive = sys.H.H_drives[1]
Hₑ_mult = a -> a[1] * X_drive

println("=== Setting up initial problems ===")
for i in 1:n_seeds
    for j in 1:n_nines
        println("Processing seed $i, fidelity target $(final_fid_floor[j])")
        
        default = UnitarySmoothPulseProblem(
            sys, U_goal, T, Δt; init_trajectory=deepcopy(def_seeds[i])
        )
        default_probs[i, j] = default
        def_fid = unitary_rollout_fidelity(default.trajectory, sys)
        init_def_fids[i,j] = def_fid
        
        add_tog_prob = UnitarySmoothPulseProblem(
            sys, U_goal, T, Δt; 
            init_trajectory=deepcopy(add_seeds[i]), 
            activate_rob_loss=true, 
            H_err=Hₑ_add, 
            Q_t=0.1
        )
        init_add_probs[i, j] = add_tog_prob
        add_fid = unitary_rollout_fidelity(add_tog_prob.trajectory, sys)
        init_add_fids[i,j] = add_fid
        
        mult_tog_prob = UnitarySmoothPulseProblem(
            sys, U_goal, T, Δt; 
            init_trajectory=deepcopy(mult_seeds[i]), 
            activate_rob_loss=true, 
            H_err=Hₑ_mult, 
            Q_t=0.1
        )
        init_mult_probs[i, j] = mult_tog_prob
        mult_fid = unitary_rollout_fidelity(mult_tog_prob.trajectory, sys)
        init_mult_fids[i,j] = mult_fid
    end
end


=== Setting up initial problems ===
Processing seed 1, fidelity target 0.99
    constructing UnitarySmoothPulseProblem...
	using integrator: DataType
	applying timesteps_all_equal constraint: Δt
    constructing UnitarySmoothPulseProblem...
	using integrator: DataType
	applying timesteps_all_equal constraint: Δt
    constructing UnitarySmoothPulseProblem...
	using integrator: DataType
	applying timesteps_all_equal constraint: Δt
Processing seed 1, fidelity target 0.9953584111663872
    constructing UnitarySmoothPulseProblem...
	using integrator: DataType
	applying timesteps_all_equal constraint: Δt
    constructing UnitarySmoothPulseProblem...
	using integrator: DataType
	applying timesteps_all_equal constraint: Δt
    constructing UnitarySmoothPulseProblem...
	using integrator: DataType
	applying timesteps_all_equal constraint: Δt
Processing seed 1, fidelity target 0.9978455653099682
    constructing UnitarySmoothPulseProblem...
	using integrator: DataType
	applying timesteps_all_equa

In [12]:
println("\n=== Initial optimization phase with resource tracking ===")
init_add_perf = Matrix{Dict}(undef, n_seeds, n_nines)
init_mult_perf = Matrix{Dict}(undef, n_seeds, n_nines)

for i in 1:n_seeds
    for j in 1:n_nines
        # Additive error optimization
        takes = 1
        total_metrics_add = SolverMetrics()
        while takes < 25 && init_add_fids[i,j] < 0.9900000
            metrics, conv_iter, wall_time, mem = track_solver_performance_detailed(
                init_add_probs[i, j]; max_iter=8, print_level=1, chunk_size=8
            )
            
            # Aggregate metrics
            append!(total_metrics_add.wall_times, metrics.wall_times)
            append!(total_metrics_add.objectives, metrics.objectives)
            append!(total_metrics_add.convergence_rates, metrics.convergence_rates)
            
            init_add_fid = unitary_rollout_fidelity(init_add_probs[i, j].trajectory, sys)
            init_add_fids[i,j] = init_add_fid
            takes += 1
        end
        init_add_perf[i,j] = analyze_convergence(total_metrics_add)
        
        # Multiplicative error optimization
        takes = 1
        total_metrics_mult = SolverMetrics()
        while takes < 25 && init_mult_fids[i,j] < 0.9900000
            metrics, conv_iter, wall_time, mem = track_solver_performance_detailed(
                init_mult_probs[i, j]; max_iter=8, print_level=1, chunk_size=8
            )
            
            # Aggregate metrics
            append!(total_metrics_mult.wall_times, metrics.wall_times)
            append!(total_metrics_mult.objectives, metrics.objectives)
            append!(total_metrics_mult.convergence_rates, metrics.convergence_rates)
            
            init_mult_fid = unitary_rollout_fidelity(init_mult_probs[i, j].trajectory, sys)
            init_mult_fids[i,j] = init_mult_fid
            takes += 1
        end
        init_mult_perf[i,j] = analyze_convergence(total_metrics_mult)
    end
end



=== Initial optimization phase with resource tracking ===
    initializing optimizer...
        applying constraint: timesteps all equal constraint
        applying constraint: initial value of Ũ⃗
        applying constraint: initial value of a
        applying constraint: final value of a
        applying constraint: bounds on a
        applying constraint: bounds on da
        applying constraint: bounds on dda
        applying constraint: bounds on Δt

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

    initializing optimizer...
        applying constraint: timesteps all equal constraint
        applying constraint: initial value of Ũ⃗
        applyi

In [13]:
# propertynames(add_probs[1, 1].objective>.L)

In [14]:
default_probs

3×10 Matrix{Any}:
 DirectTrajOptProblem
   timesteps            = 20
   duration             = 3.8
   variable names       = (:Ũ⃗, :a, :da, :dda, :Δt)
   knot point dimension = 18
  …  DirectTrajOptProblem
   timesteps            = 20
   duration             = 3.8
   variable names       = (:Ũ⃗, :a, :da, :dda, :Δt)
   knot point dimension = 18

 DirectTrajOptProblem
   timesteps            = 20
   duration             = 3.8
   variable names       = (:Ũ⃗, :a, :da, :dda, :Δt)
   knot point dimension = 18
     DirectTrajOptProblem
   timesteps            = 20
   duration             = 3.8
   variable names       = (:Ũ⃗, :a, :da, :dda, :Δt)
   knot point dimension = 18

 DirectTrajOptProblem
   timesteps            = 20
   duration             = 3.8
   variable names       = (:Ũ⃗, :a, :da, :dda, :Δt)
   knot point dimension = 18
     DirectTrajOptProblem
   timesteps            = 20
   duration             = 3.8
   variable names       = (:Ũ⃗, :a, :da, :dda, :Δt)
   knot point dimension =

In [15]:
println("\n=== Warm start optimization phase ===")
warm_add_probs = [
    UnitarySmoothPulseProblem(
        sys, U_goal, T, Δt; 
        init_trajectory=deepcopy(init_add_probs[i, j].trajectory)
    ) for i in 1:n_seeds, j in 1:n_nines
]
warm_mult_probs = [
    UnitarySmoothPulseProblem(
        sys, U_goal, T, Δt; 
        init_trajectory=deepcopy(init_mult_probs[i, j].trajectory)
    ) for i in 1:n_seeds, j in 1:n_nines
]

warm_def_perf = Matrix{Dict}(undef, n_seeds, n_nines)
warm_add_perf = Matrix{Dict}(undef, n_seeds, n_nines)
warm_mult_perf = Matrix{Dict}(undef, n_seeds, n_nines)

for i in 1:n_seeds
    for j in 1:n_nines
        println("Warm start: seed $i, fidelity target $(final_fid_floor[j])")
        
        # Default optimization
        takes = 1
        total_metrics_def = SolverMetrics()
        while takes < 150 && init_def_fids[i,j] < final_fid_floor[j]
            metrics, conv_iter, wall_time, mem = track_solver_performance_detailed(
                default_probs[i, j]; max_iter=6, print_level=1, chunk_size=6
            )
            append!(total_metrics_def.wall_times, metrics.wall_times)
            append!(total_metrics_def.objectives, metrics.objectives)
            append!(total_metrics_def.convergence_rates, metrics.convergence_rates)
            
            def_fid = unitary_rollout_fidelity(default_probs[i, j].trajectory, sys)
            init_def_fids[i, j] = def_fid
            takes += 1
        end
        warm_def_perf[i,j] = analyze_convergence(total_metrics_def)
        
        # Additive warm start
        takes = 1
        total_metrics_add = SolverMetrics()
        while takes < 50 && init_add_fids[i,j] < final_fid_floor[j]
            metrics, conv_iter, wall_time, mem = track_solver_performance_detailed(
                warm_add_probs[i, j]; max_iter=10, print_level=1, chunk_size=10
            )
            append!(total_metrics_add.wall_times, metrics.wall_times)
            append!(total_metrics_add.objectives, metrics.objectives)
            append!(total_metrics_add.convergence_rates, metrics.convergence_rates)
            
            warm_add_fid = unitary_rollout_fidelity(warm_add_probs[i, j].trajectory, sys)
            init_add_fids[i,j] = warm_add_fid
            takes += 1
        end
        warm_add_perf[i,j] = analyze_convergence(total_metrics_add)
        
        # Multiplicative warm start
        takes = 1
        total_metrics_mult = SolverMetrics()
        while takes < 50 && init_mult_fids[i,j] < final_fid_floor[j]
            metrics, conv_iter, wall_time, mem = track_solver_performance_detailed(
                warm_mult_probs[i, j]; max_iter=10, print_level=1, chunk_size=10
            )
            append!(total_metrics_mult.wall_times, metrics.wall_times)
            append!(total_metrics_mult.objectives, metrics.objectives)
            append!(total_metrics_mult.convergence_rates, metrics.convergence_rates)
            
            warm_mult_fid = unitary_rollout_fidelity(warm_mult_probs[i, j].trajectory, sys)
            init_mult_fids[i,j] = warm_mult_fid
            takes += 1
        end
        warm_mult_perf[i,j] = analyze_convergence(total_metrics_mult)
    end
end




=== Warm start optimization phase ===
    constructing UnitarySmoothPulseProblem...
	using integrator: DataType
	applying timesteps_all_equal constraint: Δt
    constructing UnitarySmoothPulseProblem...
	using integrator: DataType
	applying timesteps_all_equal constraint: Δt
    constructing UnitarySmoothPulseProblem...
	using integrator: DataType
	applying timesteps_all_equal constraint: Δt
    constructing UnitarySmoothPulseProblem...
	using integrator: DataType
	applying timesteps_all_equal constraint: Δt
    constructing UnitarySmoothPulseProblem...
	using integrator: DataType
	applying timesteps_all_equal constraint: Δt
    constructing UnitarySmoothPulseProblem...
	using integrator: DataType
	applying timesteps_all_equal constraint: Δt
    constructing UnitarySmoothPulseProblem...
	using integrator: DataType
	applying timesteps_all_equal constraint: Δt
    constructing UnitarySmoothPulseProblem...
	using integrator: DataType
	applying timesteps_all_equal constraint: Δt
    const

In [16]:
println("\n=== Final MaxToggle optimization phase ===")
n_nines = length(final_fid_floor)
final_add_probs = Matrix{Any}(undef, n_seeds, n_nines)
final_mult_probs = Matrix{Any}(undef, n_seeds, n_nines)
final_add_perf = Matrix{Dict}(undef, n_seeds, n_nines)
final_mult_perf = Matrix{Dict}(undef, n_seeds, n_nines)

for i in 1:n_seeds
    for j in 1:n_nines
        println("MaxToggle: seed $i, fidelity target $(final_fid_floor[j])")
        
        # Additive MaxToggle
        Hₑ_add = a -> PAULIS.X
        add_prob = UnitaryMaxToggleProblem(
            warm_add_probs[i,j],
            U_goal,
            Hₑ_add;
            Q_t=1.0,
            final_fidelity=final_fid_floor[j],
            piccolo_options=piccolo_opts
        )
        
        metrics_add, conv_iter_add, wall_time_add, mem_add = track_solver_performance_detailed(
            add_prob; max_iter=150, print_level=5, chunk_size=10
        )
        final_add_probs[i,j] = add_prob
        final_add_perf[i,j] = analyze_convergence(metrics_add)
        
        # Multiplicative MaxToggle
        X_drive = sys.H.H_drives[1]
        Hₑ_mult = a -> a[1] * X_drive
        
        mult_prob = UnitaryMaxToggleProblem(
            warm_mult_probs[i,j],
            U_goal,
            Hₑ_mult;
            Q_t=1.0,
            final_fidelity=final_fid_floor[j],
            piccolo_options=piccolo_opts
        )
        
        metrics_mult, conv_iter_mult, wall_time_mult, mem_mult = track_solver_performance_detailed(
            mult_prob; max_iter=150, print_level=5, chunk_size=10
        )
        final_mult_probs[i,j] = mult_prob
        final_mult_perf[i,j] = analyze_convergence(metrics_mult)
    end
end



=== Final MaxToggle optimization phase ===
MaxToggle: seed 1, fidelity target 0.99
    initializing optimizer...
        applying constraint: timesteps all equal constraint
        applying constraint: initial value of Ũ⃗
        applying constraint: initial value of a
        applying constraint: final value of a
        applying constraint: bounds on a
        applying constraint: bounds on da
        applying constraint: bounds on dda
        applying constraint: bounds on Δt
        applying constraint: timesteps all equal constraint
This is Ipopt version 3.14.19, running with linear solver MUMPS 5.8.1.

Number of nonzeros in equality constraint Jacobian...:     2414
Number of nonzeros in inequality constraint Jacobian.:        8
Number of nonzeros in Lagrangian Hessian.............:    24593

Total number of variables............................:      346
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      19

Excessive output truncated after 524346 bytes.

In [ ]:
println("\n=== Generating performance analysis ===")

# Create summary DataFrame
function create_performance_summary(perf_matrix, prob_type)
    df_data = []
    for i in 1:n_seeds
        for j in 1:n_nines
            if haskey(perf_matrix[i,j], "total_wall_time") && perf_matrix[i,j]["total_wall_time"] > 0
                push!(df_data, (
                    seed = i,
                    fidelity_target = final_fid_floor[j],
                    problem_type = prob_type,
                    wall_time = perf_matrix[i,j]["total_wall_time"],
                    avg_iter_time = isnan(perf_matrix[i,j]["avg_iteration_time"]) ? 0.0 : perf_matrix[i,j]["avg_iteration_time"],
                    convergence_iter = isnothing(perf_matrix[i,j]["convergence_iteration"]) ? missing : perf_matrix[i,j]["convergence_iteration"],
                    total_iterations = perf_matrix[i,j]["total_iterations"],
                    peak_memory_mb = perf_matrix[i,j]["peak_memory_mb"],
                    final_objective = isnan(perf_matrix[i,j]["final_objective"]) ? missing : perf_matrix[i,j]["final_objective"]
                ))
            end
        end
    end
    return isempty(df_data) ? DataFrame() : DataFrame(df_data)
end

# Generate summary tables
df_warm_def = create_performance_summary(warm_def_perf, "Default")
df_warm_add = create_performance_summary(warm_add_perf, "Warm_Additive")
df_warm_mult = create_performance_summary(warm_mult_perf, "Warm_Multiplicative")
df_final_add = create_performance_summary(final_add_perf, "MaxToggle_Additive")
df_final_mult = create_performance_summary(final_mult_perf, "MaxToggle_Multiplicative")

# Combine all DataFrames
df_all = vcat(df_warm_def, df_warm_add, df_warm_mult, df_final_add, df_final_mult)
       
# Replace nothing values with missing for CSV compatibility
for col in names(df_all)
    df_all[!, col] = replace(df_all[!, col], nothing => missing)
end

# Save to CSV for further analysis
CSV.write("solver_performance_metrics.csv", df_all)


"solver_performance_metrics.csv"

In [31]:
# Plot 1: Wall Clock Time vs Fidelity Target
fig1 = Figure(resolution = (1200, 800))
ax1 = Axis(fig1[1, 1], 
    title = "Wall Clock Time vs Fidelity Target",
    xlabel = "Target Fidelity",
    ylabel = "Wall Clock Time (s)",
    xscale = log10
)

for prob_type in unique(df_all.problem_type)
    df_subset = filter(row -> row.problem_type == prob_type, df_all)
    CairoMakie.scatter!(ax1, 1 .- df_subset.fidelity_target, df_subset.wall_time, 
             label = prob_type, markersize = 10)
end
axislegend(ax1)
save("wall_time_vs_fidelity.png", fig1)

# Plot 2: Convergence Rate Analysis
fig2 = Figure(resolution = (1200, 800))
ax2 = Axis(fig2[1, 1],
    title = "Convergence Iteration vs Fidelity Target",
    xlabel = "Target Fidelity",
    ylabel = "Iterations to Convergence",
    xscale = log10
)

for prob_type in unique(df_all.problem_type)
    df_subset = filter(row -> row.problem_type == prob_type && 
                       !ismissing(row.convergence_iter) && 
                       !isnothing(row.convergence_iter), df_all)

    if nrow(df_subset) > 0
        CairoMakie.scatter!(ax2, 1 .- df_subset.fidelity_target, df_subset.convergence_iter,
                 label = prob_type, markersize = 10)
    end
    
end
axislegend(ax2)
save("convergence_iteration_vs_fidelity.png", fig2)

# Plot 3: Memory Usage Analysis
fig3 = Figure(resolution = (1200, 800))
ax3 = Axis(fig3[1, 1],
    title = "Peak Memory Usage vs Fidelity Target",
    xlabel = "Target Fidelity", 
    ylabel = "Peak Memory (MB)",
    xscale = log10
)

for prob_type in unique(df_all.problem_type)
    df_subset = filter(row -> row.problem_type == prob_type, df_all)
    CairoMakie.scatter!(ax3, 1 .- df_subset.fidelity_target, df_subset.peak_memory_mb,
             label = prob_type, markersize = 10)
end
axislegend(ax3)
save("memory_usage_vs_fidelity.png", fig3)

# Plot 4: Time per Iteration
fig4 = Figure(resolution = (1200, 800))
ax4 = Axis(fig4[1, 1],
    title = "Average Time per Iteration vs Fidelity Target",
    xlabel = "Target Fidelity",
    ylabel = "Time per Iteration (s)",
    xscale = log10
)

for prob_type in unique(df_all.problem_type)
    df_subset = filter(row -> row.problem_type == prob_type, df_all)
    CairoMakie.scatter!(ax4, 1 .- df_subset.fidelity_target, df_subset.avg_iter_time,
             label = prob_type, markersize = 10)
end
axislegend(ax4)
save("time_per_iteration_vs_fidelity.png", fig4)



In [25]:
println("\n=== Performance Summary Statistics ===")
for prob_type in unique(df_all.problem_type)
    df_subset = filter(row -> row.problem_type == prob_type, df_all)
    if nrow(df_subset) > 0
        println("\n$prob_type:")
        
        # Filter out NaN and missing values for statistics
        valid_wall_times = filter(!isnan, df_subset.wall_time)
        valid_iterations = filter(!isnan, df_subset.total_iterations)
        valid_iter_times = filter(x -> !isnan(x) && !ismissing(x), df_subset.avg_iter_time)
        valid_memory = filter(!isnan, df_subset.peak_memory_mb)
        
        if !isempty(valid_wall_times)
            println("  Average wall time: $(mean(valid_wall_times)) ± $(std(valid_wall_times)) s")
        end
        if !isempty(valid_iterations)
            println("  Average iterations: $(mean(valid_iterations)) ± $(std(valid_iterations))")
        end
        if !isempty(valid_iter_times)
            println("  Average time/iter: $(mean(valid_iter_times)) ± $(std(valid_iter_times)) s")
        end
        if !isempty(valid_memory)
            println("  Average memory: $(mean(valid_memory)) ± $(std(valid_memory)) MB")
        end
        
        conv_iters = filter(x -> !isnothing(x) && !ismissing(x), df_subset.convergence_iter)
        if !isempty(conv_iters)
            conv_iters_numeric = [x for x in conv_iters if isa(x, Number)]
            if !isempty(conv_iters_numeric)
                println("  Average convergence: $(mean(conv_iters_numeric)) ± $(std(conv_iters_numeric)) iterations")
            end
        end
    end
end


In [20]:
println("\n=== Detailed Convergence Analysis ===")

# Function to analyze convergence threshold sensitivity
function analyze_convergence_thresholds(perf_matrix, thresholds=[1e-4, 1e-5, 1e-6, 1e-7])
    results = Dict()
    for threshold in thresholds
        conv_iters = []
        for i in 1:size(perf_matrix, 1)
            for j in 1:size(perf_matrix, 2)
                if haskey(perf_matrix[i,j], "convergence_rates")
                    rates = perf_matrix[i,j]["convergence_rates"]
                    conv_iter = findfirst(r -> r < threshold, rates)
                    if !isnothing(conv_iter)
                        push!(conv_iters, conv_iter)
                    end
                end
            end
        end
        results[threshold] = conv_iters
    end
    return results
end

# Analyze for different convergence thresholds
println("\nConvergence at different thresholds:")
for prob_type in ["MaxToggle_Additive", "MaxToggle_Multiplicative"]
    perf_matrix = prob_type == "MaxToggle_Additive" ? final_add_perf : final_mult_perf
    
    println("\n$prob_type:")
    for threshold in [1e-4, 1e-5, 1e-6, 1e-7]
        conv_analysis = analyze_convergence_thresholds(perf_matrix, [threshold])
        if !isempty(conv_analysis[threshold])
            avg_conv = mean(conv_analysis[threshold])
            println("  Threshold $threshold: avg convergence at iteration $(avg_conv)")
        end
    end
end

println("\n=== Analysis Complete ===")
println("Results saved to:")
println("  - solver_performance_metrics.csv")
println("  - wall_time_vs_fidelity.png")
println("  - convergence_iteration_vs_fidelity.png")
println("  - memory_usage_vs_fidelity.png")
println("  - time_per_iteration_vs_fidelity.png")